In [1]:
import sys
import torch
sys.path.append("../")

from giung2.config import get_cfg
from giung2.data.build import build_dataloaders
from giung2.modeling.build import build_model
from giung2.solver.build import build_optimizer, build_scheduler
from giung2.evaluation import evaluate_acc, evaluate_nll, evaluate_ece

In [2]:
# load config file
cfg = get_cfg()
cfg.merge_from_file("../configs/CIFAR_1GPU/C10_WRN28x1-BN-ReLU_SGD.yaml")

In [3]:
# build model
model = build_model(cfg)
model.cuda()

ClassificationModelBase(
  (backbone): PreResNet(
    (layers): Sequential(
      (0): FirstBlock(
        (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (norm1): Identity()
        (relu1): Identity()
        (pool1): Identity()
      )
      (1): BasicBlock(
        (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU()
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (shortcut): Identity()
      )
      (2): BasicBlock(
        (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [4]:
# build dataloaders
dataloaders = build_dataloaders(cfg, root="../datasets/")
dataloaders.keys()

dict_keys(['dataloader', 'trn_loader', 'val_loader', 'tst_loader'])

In [5]:
# build optimizer and scheduler
optimizer = build_optimizer(cfg, model)
scheduler = build_scheduler(cfg, optimizer)

In [6]:
# now, we can train the model
for epoch_idx, _ in enumerate(range(cfg.SOLVER.NUM_EPOCHS), start=1):

    if epoch_idx == 3:
        break

    model.train()

    losses = []
    for batch_idx, (images, labels) in enumerate(dataloaders["dataloader"], start=1):
        outputs = model(images, labels)
        loss = torch.nn.functional.cross_entropy(outputs["logits"], outputs["labels"])
        losses.append(loss.detach().item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 44 == 0:
            print(f"Epoch {epoch_idx:3d}, Batch {batch_idx:3d}: average loss = {sum(losses) / len(losses):.4f}")
    
    scheduler.step()

Epoch   1, Batch  44: average loss = 2.4975
Epoch   1, Batch  88: average loss = 2.3353
Epoch   1, Batch 132: average loss = 2.2399
Epoch   1, Batch 176: average loss = 2.1791
Epoch   1, Batch 220: average loss = 2.1322
Epoch   1, Batch 264: average loss = 2.0923
Epoch   1, Batch 308: average loss = 2.0623
Epoch   1, Batch 352: average loss = 2.0335
Epoch   2, Batch  44: average loss = 1.7987
Epoch   2, Batch  88: average loss = 1.7300
Epoch   2, Batch 132: average loss = 1.6837
Epoch   2, Batch 176: average loss = 1.6388
Epoch   2, Batch 220: average loss = 1.6004
Epoch   2, Batch 264: average loss = 1.5615
Epoch   2, Batch 308: average loss = 1.5268
Epoch   2, Batch 352: average loss = 1.4956


In [7]:
# after training, we want to evaluate the model
model.eval()

pred_logits_list = []
true_labels_list = []
for images, labels in dataloaders["tst_loader"]:
    with torch.no_grad():
        outputs = model(images)
    pred_logits_list.append(outputs["logits"].cpu())
    true_labels_list.append(labels)

pred_logits_list = torch.cat(pred_logits_list)
true_labels_list = torch.cat(true_labels_list)
confidences_list = torch.softmax(pred_logits_list, dim=1)

print(f"ACC = {evaluate_acc(confidences_list, true_labels_list):.4f}")
print(f"NLL = {evaluate_nll(confidences_list, true_labels_list):.4f}")
print(f"ECE = {evaluate_ece(confidences_list, true_labels_list):.4f}")

ACC = 0.5086
NLL = 1.3435
ECE = 0.0515
